<a href="https://colab.research.google.com/github/Leinadh/PeruvianImageGenerator/blob/master/face_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# !tar -zxvf '/content/drive/My Drive/Made with ML/datasets/vgg_face_dataset.tar.gz'

# Image Preprocessing

In [17]:
!rm -r PeruvianImageGenerator/ # to delete file cloned before

In [6]:
!git clone https://github.com/Leinadh/PeruvianImageGenerator.git

Cloning into 'PeruvianImageGenerator'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 14395 (delta 0), reused 2 (delta 0), pack-reused 14391
Receiving objects: 100% (14395/14395), 595.15 MiB | 18.10 MiB/s, done.
Resolving deltas: 100% (21/21), done.
Checking out files: 100% (14324/14324), done.


In [7]:
import PeruvianImageGenerator.scripts.download_faces as dlf
import PeruvianImageGenerator.scripts.plot_utils as plot_utils

In [18]:
data_path = '/content/PeruvianImageGenerator/datasets/face_datasets/vgg_face_dataset/files'
target_path='/content/tmp/face_images/'
# dlf.parse_data(data_path, 1000, 1, target_path, 10, 40)
dlf.download_vgg_images(data_path, num_people=2622, num_images=1, target_path=target_path,
                        offset_x_percent=25, offset_top_percent=65,
                        offset_bottom_percent=12, min_pose=3, min_score=0,
                        curation=False, formats_allowed=['jpg', 'jpeg'], from_notebook=True)

/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


In [20]:
dlf.clean_corrupt_files(target_path, formats_allowed=['jpg', 'jpeg'])

Removing corrupt file: Dean_Martin_1.jpg
Removing corrupt file: Jim_Sturgess_1.jpg
Removing corrupt file: Diana_Rigg_1.jpg
Removing corrupt file: Ida_Lupino_1.jpg
Removing corrupt file: Alexander_Siddig_1.jpg
Removing corrupt file: Gloria_Reuben_1.jpg
Removing corrupt file: Abel_Ferrara_1.jpg
Removing corrupt file: James_Stewart_1.jpg
Removing corrupt file: Marcello_Mastroianni_1.jpg
Removing corrupt file: Dwight_Schultz_1.jpg
Removing corrupt file: William_Devane_1.jpg
Removing corrupt file: Christina_Moore_1.jpg
Removing corrupt file: Andy_Richter_1.jpg
Removing corrupt file: Lee_Marvin_1.jpg
Removing corrupt file: John_Amos_1.jpg
Removing corrupt file: Fred_Astaire_1.jpg
Removing corrupt file: Jane_Campion_1.jpg
Removing corrupt file: Adam_Rodriguez_1.jpg
Removing corrupt file: Jeffrey_Hunter_1.jpg
Removing corrupt file: Joey_Heatherton_1.jpg
Removing corrupt file: Sal_Mineo_1.jpg
Removing corrupt file: Jack_Lemmon_1.jpg
Removing corrupt file: Grace_Zabriskie_1.jpg
Removing corrupt 

In [22]:
# !zip -r /content/face_images.zip tmp/face_images

# Image Segmentation

In [1]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image

In [ ]:
# See random image

path = '/content/tmp'
img_size = 64
batch_size = 64
# Especificamos la función de preprocesamiento
train_datagen = ImageDataGenerator()
train_generator = train_datagen.flow_from_directory(path, target_size=(img_size, img_size), batch_size=batch_size, subset='training')
# Veamos alguna de las imágenes
random_img = np.random.choice(train_generator.filenames)
print(random_img)
Image.open(path + '/' + random_img)

In [ ]:
!pip install keras-segmentation

In [69]:
from keras_segmentation.pretrained import pspnet_101_voc12
import cv2
import os
from tqdm.notebook import tqdm
from time import time

In [64]:
def remove_background(img_mask, img1, output_path, output_file):

  seg_gray = cv2.cvtColor(img_mask, cv2.COLOR_BGR2GRAY)
  _,bg_mask = cv2.threshold(seg_gray, 0, 255, cv2.THRESH_BINARY_INV|cv2.THRESH_OTSU)

  ## convert mask to 3-channels
  bg_mask = cv2.cvtColor(bg_mask, cv2.COLOR_GRAY2BGR)

  ## cv2.bitwise_and to extract the region
  bg = cv2.bitwise_or(img1, bg_mask)

  ## save 
  cv2.imwrite(output_path + output_file, bg)

In [ ]:
#remove background
path = '/content/tmp/face_images/'
mask_path = '/content/tmp/masks/'
output_path = '/content/face_images_wo_bg/'

if not os.path.exists(mask_path):
  os.makedirs(mask_path)

if not os.path.exists(output_path):
  os.makedirs(output_path)

model = pspnet_101_voc12() # load the pretrained model trained on Pascal VOC 2012 dataset

start = time()
for filename in tqdm(os.listdir(path)):
  # filename = filename.split('/')[1] 
  mask_file = filename[:-4] + "_mask.png"
  output_file = filename[:-4] + "_wo_bg.jpg"


  out = model.predict_segmentation(
    inp= path + filename,
    out_fname= mask_path + mask_file
  )
  
  img_mask = cv2.imread(mask_path + mask_file)
  img1 = cv2.imread(path + filename) #READ BGR

  remove_background(img_mask, img1, output_path, output_file)
end = time()
print(f'Processing time: {round((end-start)/60, 2)} min\n')

In [ ]:
!zip -r -q /content/face_images_wo_bg.zip face_images_wo_bg

In [ ]:
!cp -r /content/face_images_wo_bg.zip "/content/drive/My Drive/Made with ML/datasets/"